# AlexNet
In this, I will be attempting to reimplement AlexNet using PyTorch. The data loading part of this implementation is taken from https://github.com/dansuh17/alexnet-pytorch/blob/master/model.py

In [1]:
# importing torch and torchvision methods
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter


# modules for dataset transformation/processing
from torch import nn, optim
from tensorboardX import SummaryWriter
from torchvision.datasets import CIFAR10
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision import transforms
from torchvision.utils import save_image

# importing basic plotting + computation libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# file management library
import os

# checking if GPU available for training
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
# defining constants
NUM_EPOCHS = 90
BATCH_SIZE = 128

MOMENTUM = 0.9
LR_DECAY = 0.0005
LR_INIT = 0.01

IMAGE_DIM = 227
NUM_CLASSES = 1000  # 1000 classes for imagenet 2012 dataset

# modify this to point to your data directory
INPUT_ROOT_DIR = 'alexnet_data_in'
TRAIN_IMG_DIR = 'alexnet_data_in/imagenet'
OUTPUT_DIR = 'alexnet_data_out'
LOG_DIR = OUTPUT_DIR + '/tblogs'  # tensorboard logs
CHECKPOINT_DIR = OUTPUT_DIR + '/models'  # model checkpoints

In [3]:
# defining the network structure
class AlexNet(nn.Module):
    
    def __init__(self, num_classes=1000):
        """
        
        """
    
    
        # calling parent constructor        
        super().__init__()
        
        # compsition of convolutional + maxpooling + activation layers
        self.extract_features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, stride=4, kernel_size=11, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2, stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        # composition of linear classification layers
        self.MLP = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features=256 * 6 * 6, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=1000)
        )
        
        self.init_bias()
        
        # defining the network in terms of individual layers
        """
        self.cnv1 = nn.Conv2d(in_channels=3, out_channels=96, stride=4, kernel_size=11, padding=0)
        self.mp = nn.MaxPool2d(kernel_size=3, stride=2)
        self.cnv2 = nn.Conv2d(in_channels=96, out_channels=256,kernel_size=5, padding=2, stride=1)
        self.cnv3 = nn.Conv2d(in_channels=256, out_channels=384,kernel_size=3, padding=1, stride=1)
        self.cnv4 = nn.Conv2d(in_channels=384, out_channels=384,kernel_size=3, padding=1, stride=1)
        self.cnv5 = nn.Conv2d(in_channels=384, out_channels=256,kernel_size=3, padding=1, stride=1)
        self.fc1 = nn.Linear(in_features=9216, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=10)
        """
        
    def forward(self, x):
        
        x = self.extract_features(x)
        x = x.view(-1, 256 *6 * 6)
        x = self.MLP(x)
        
        return x
        
        # original definition of the forward propogation with individual layer defs
        """
        x = F.relu(self.conv1(x))
        x = self.mp(x)
        x = F.relu(self.conv2(x))
        x = self.mp(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.mp(x)
        
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x
        """
        
    def init_bias(self):
        for layer in self.extract_features:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.extract_features[4].bias, 1)
        nn.init.constant_(self.extract_features[10].bias, 1)
        nn.init.constant_(self.extract_features[12].bias, 1)

In [4]:
# loading the dataset


# getting an initial seed
seed = torch.initial_seed()

tbwriter = SummaryWriter(log_dir=LOG_DIR)

# loading the model
alexnet = AlexNet(num_classes=NUM_CLASSES).to(device)

# using dataloaders
dataset = datasets.ImageFolder(TRAIN_IMG_DIR, transforms.Compose([
    transforms.CenterCrop(IMAGE_DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

dataloader = data.DataLoader(
    dataset,
    shuffle=True,
    pin_memory=True,
    num_workers=8,
    drop_last=True,
    batch_size=BATCH_SIZE
)

In [5]:
# creating the optimizer
alexnet = AlexNet()
optimizer = optim.Adam(params=alexnet.parameters(), lr=1e-4)

In [6]:
# model training
total_steps=1
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(NUM_EPOCHS):
    lr_scheduler.step()
    for imgs, classes in dataloader:
        imgs, classes = imgs.to(device), classes.to(device)
        
        output=alexnet(imgs)
        loss=F.cross_entropy(output, classes)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        if total_steps % 10 == 0:
            with torch.no_grad():
                _, preds = torch.max(output, 1)
                accuracy = torch.sum(preds == classes)
                
            print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'
                            .format(epoch + 1, total_steps, loss.item(), accuracy.item()))
            tbwriter.add_scalar('loss', loss.item(), total_steps)
            tbwriter.add_scalar('accuracy', accuracy.item(), total_steps)
        
        if total_steps % 100 == 0:
                with torch.no_grad():
                    # print and save the grad of the parameters
                    # also print and save parameter values
                    print('*' * 10)
                    for name, parameter in alexnet.named_parameters():
                        if parameter.grad is not None:
                            avg_grad = torch.mean(parameter.grad)
                            print('\t{} - grad_avg: {}'.format(name, avg_grad))
                            tbwriter.add_scalar('grad_avg/{}'.format(name), avg_grad.item(), total_steps)
                            tbwriter.add_histogram('grad/{}'.format(name),
                                    parameter.grad.cpu().numpy(), total_steps)
                        if parameter.data is not None:
                            avg_weight = torch.mean(parameter.data)
                            print('\t{} - param_avg: {}'.format(name, avg_weight))
                            tbwriter.add_histogram('weight/{}'.format(name),
                                    parameter.data.cpu().numpy(), total_steps)
                            tbwriter.add_scalar('weight_avg/{}'.format(name), avg_weight.item(), total_steps)
        
        total_steps += 1
        
    checkpoint_path = os.path.join(CHECKPOINT_DIR, 'alexnet_states_e{}.pkl'.format(epoch + 1))
    state = {
        'epoch': epoch,
        'total_steps': total_steps,
        'optimizer': optimizer.state_dict(),
        'model': alexnet.state_dict(),
        'seed': seed,
    }
    torch.save(state, checkpoint_path)

/home/sbaveja/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: 1 	Step: 10 	Loss: 5.2051 	Acc: 2
Epoch: 1 	Step: 20 	Loss: 4.9762 	Acc: 2
Epoch: 1 	Step: 30 	Loss: 4.8682 	Acc: 1
Epoch: 1 	Step: 40 	Loss: 4.9165 	Acc: 0
Epoch: 1 	Step: 50 	Loss: 4.8508 	Acc: 1
Epoch: 1 	Step: 60 	Loss: 4.8300 	Acc: 1
Epoch: 1 	Step: 70 	Loss: 4.8489 	Acc: 2
Epoch: 1 	Step: 80 	Loss: 4.8593 	Acc: 1
Epoch: 1 	Step: 90 	Loss: 4.8713 	Acc: 0
Epoch: 1 	Step: 100 	Loss: 4.8782 	Acc: 0
**********
	extract_features.0.weight - grad_avg: 5.494367997016525e-06
	extract_features.0.weight - param_avg: -0.00045165038318373263
	extract_features.0.bias - grad_avg: -2.3085005977918627e-06
	extract_features.0.bias - param_avg: 0.0004981197998858988
	extract_features.4.weight - grad_avg: 2.3501314672103035e-07
	extract_features.4.weight - param_avg: -0.0007400429458357394
	extract_features.4.bias - grad_avg: 1.7282565067944233e-06
	extract_features.4.bias - param_avg: 0.9992239475250244
	extract_features.8.weight - grad_avg: 3.1269598821381805e-07
	extract_features.8.weight -

FileNotFoundError: [Errno 2] No such file or directory: 'alexnet_data_out/models/alexnet_states_e1.pkl'